ABSA

In [1]:
#LIBRERIE
import spacy
import regex as re
import pandas as pd
from copy import deepcopy
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
analyzer = SentimentIntensityAnalyzer()
from pandas_ods_reader import read_ods
nlp = spacy.load('it_core_news_sm')

In [2]:
#load data
df = read_ods("E:\Tesi Magistrale 2023\datasets\input_finali\input_1_sent_finale_v2.ods")

In [3]:
df

,user_id,screen_name,favorite_count,total_args,ANIMALI OR CUCCIOLI OR PICCOLI AND PRESI OR PORTATI,GUERRA AND UCRAINA,SOSTENIBILITà OR AUTO_ELETTRICHE OR AUTO_ELETTRICA,VACCINI OR VAX OR NO_VAX OR SI_VAX,FRATELLI AND ITALIA OR FDI,FABIO_FAZIO OR FAZIO OR CHE_TEMPO_CHE_FA OR CHE_TEMPO_CHE_FU,...,G7 object_all,ALBERTO_ANGELA OR DIVULGAZIONE_SCIENTIFICA object_all,CEMENTIFICAZIONE OR OVIDIO_MARRAS object_all,FAMIGLIA_TRADIZIONALE object_all,FIORAVANTI OR VALERIO_FIORAVANTI OR STRAGE_DI_BOLOGNA object_all,2_GIUGNO OR FESTA_DELLA_REPUBBLICA object_all,MES OR MECCANISMO_EUROPEO_DI_STABILITà object_all,SALUTO_ROMANO OR DECIMA_MAS OR XMAS OR X_MAS object_all,MASSIMO_ZEN OR LEGITTIMA_DIFESA AND UCCISO OR UCCIDERE object_all,ABORTO OR INTERRUZIONE_DI_GRAVIDANZA object_all
0,1392726045082767360,fratotolo2,1879.0,18.0,0.0,0.0,1.0,1.0,0.0,1.0,...,NA,NA,NA,Come spiegare la distruzione della civiltà occ...,NA,Scontro tra giornalisti di #Repubblica sul “sa...,NA,"Cazzarola @PBerizzi, altra figura di palta: no...",NA,NA
1,900766265245130753,LaVeritaWeb,1761.0,13.0,0.0,0.0,1.0,1.0,0.0,1.0,...,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
2,1434022273506324482,itsmeback_,2688.0,12.0,0.0,1.0,0.0,0.0,0.0,1.0,...,NA,NA,NA,NA,Giusva Fioravanti.\nAssassino di 95 persone.\n...,NA,NA,NA,NA,NA
3,1077691849,Misurelli77,866.0,11.0,0.0,0.0,0.0,0.0,0.0,1.0,...,NA,NA,NA,NA,L'unità di Sansonetti(direttore) chiama\nil te...,NA,NA,NA,NA,NA
4,1557514573,elio_vito,7581.0,11.0,0.0,0.0,1.0,0.0,1.0,0.0,...,NA,NA,NA,Ieri i potenti della destra hanno sfoggiato le...,NA,NA,NA,NA,NA,NA
5,1683455144,ImolaOggi,660.0,11.0,0.0,0.0,1.0,1.0,0.0,1.0,...,"G7, Biden: ""Libereremo il mondo dalle armi nuc...",NA,NA,NA,NA,NA,NA,NA,NA,NA
6,827344747,siriomerenda,3414.0,11.0,0.0,0.0,0.0,0.0,1.0,1.0,...,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
7,987756923129876481,PBerizzi,2547.0,11.0,0.0,0.0,0.0,0.0,1.0,1.0,...,NA,NA,NA,NA,Fioravanti sull'Unità può sembrare una delle t...,Buona festa della Repubblica nata dalla Resist...,NA,Il saluto romano sotto la bandiera nazista del...,NA,NA
8,1061519784079171584,ilruttosovrano,3045.0,9.0,0.0,0.0,0.0,0.0,1.0,1.0,...,NA,NA,NA,NA,NA,NA,NA,"No, non era un saluto romano, ma La Russa non ...",NA,NA
9,1323040160,strange_days_82,4087.0,9.0,0.0,1.0,0.0,0.0,0.0,1.0,...,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA


Preparare il dataset:

In [4]:
def cleantxt(x):
    if isinstance(x, str):  # Verifica se il valore è di tipo stringa
        res = x
        res = re.sub(r'<.*?>', '', res)  # Rimuovi i tag HTML
        res = re.sub(r'https?://\S+|www\.\S+', '', res)  # Rimuovi gli URL
        res = re.sub('\s+', ' ', res)  # Rimuovi gli spazi vuoti
        res = re.sub('@', '', res)  # Rimuovi il simbolo @
        res = re.sub('#', '', res)  # Rimuovi il simbolo #
        emoji_char = re.compile("[" 
                               u"\U0001F600-\U0001F64F"  # Emoticon
                               u"\U0001F300-\U0001F5FF"  # Simboli e pictogrammi
                               u"\U0001F680-\U0001F6FF"  # Simboli di trasporto e mappe
                               u"\U0001F1E0-\U0001F1FF"  # Bandiere (iOS)
                               u"\U00002500-\U00002BEF"  # Caratteri cinesi
                               u"\U00002702-\U000027B0"
                               u"\U00002702-\U000027B0"
                               u"\U000024C2-\U0001F251"
                               u"\U0001f926-\U0001f937"
                               u"\U00010000-\U0010ffff"
                               u"\u2640-\u2642"
                               u"\u2600-\u2B55"
                               u"\u200d"
                               u"\u23cf"
                               u"\u23e9"
                               u"\u231a"
                               u"\ufe0f"  # Dingbats
                               u"\u3030"
                               "]+", flags=re.UNICODE)
        res = emoji_char.sub(r'', res)  # Rimuovi emoticon, pictogrammi e altri caratteri insoliti
        res = re.sub("&amp", "and", res).replace("\n", " ")  # Rimuovi i rifiuti dovuti alla codifica
        return res
    else:
        return x  # Restituisci il valore originale se non è una stringa

In [5]:
df.drop(197, inplace=True)

In [5]:
#remove probelmatic rows
col_drop = [1639,4525]
df.drop(col_drop, inplace=True)

LOOP PER ASSEGNARE OPINION POLARITIES:


In [5]:
#crea lista con tutti i pattern in fila 
rgx = [
    r"(\b(ANIMAL|CUCCIOL|PICCOL)\w*).*(\b(PRES|PORTAT)\w*)" ,
    r"(.*\b(GUERRA)\b)|(.*\b(UCRAINA)\b)",
   r"(?i).*\b(?:SOSTENIBIL\w+|AUT\w*\b.*\bELETTRIC\w+)\b.*",
    r"(\b(VACCIN\w*)|.*?(SI[\s-]?VAX))",
    r"\bFRATELLI\b|\bITALIA\b|\bFDI\b.*",
    r"(\bFABIO FAZIO\b)|(\bFAZIO\b)|(\bCHE TEMPO CHE F\w)",
    r".*\b(?:EUROVISION\w*|\w*EUROVISION\w*)\b.*",
    r"(\bMISUR\w*\s*(?:ANTI[\s-]?COVID))|((\bGREEN\s?PASS\b)|(\bGREEN\w?PASS))|\bTAMPON\w*",
    r"(\bLGBT[\w+]*\b)",
    r"(\bGIORGIA MELONI\b)|(\bMELONI\b)",
    r"(\bCARLO ROVELLI\b)|(\bROVELLI\b)",
    r"(\bIKRAM JARMOUNI\b)|(\bJARMOUNI\b)",
    r"(\bRAZZIS\w.*)",
    r"(\bFEMMINIS\w*)|(.*\bVIOLEN\w.*?)|(.\\bDONN\w.*)|(\bFEMMINICIDIO\b)|(\bGIULIA TRAMONTANO\b)",
    r"(\bMATTEO SALVINI\b)|(\bSALVINI\b)",
    r"(\b(GIOVANI|RAGAZZI|UNIVERSITARI|STUDENTI).*?(AFFITT\w.|TEND\w.))",
    r"(\bOMOFOB\w*)",
    r"(?i)\b.*ALLUVION\w*|\b(?:EMILIAROMAGNA|EMILIA[-\s]?ROMAGNA|EMILIA|ROMAGNA)\b.*",
    r"(\bCAMBIAMENT\w.*?(\bCLIMATIC\w*|\bAMBIENTAL\w.*))",
    r"\bATTIVIST\w.*|\bAMBIENT\w*|(\bULTIMA GENERAZIONE\b)|(\bEXTINCTION REBELLION\b)|(\bFRIDAYS FOR FUTURE\b)|(\bFRIDAYSFORFUTURE\b)",
    r"(\bLAVOR\w.*(\\bGIOVAN\w.*|\\bRAGAZZ\w.*?))",
    r"\bPOLIZIA\b.*(\bVIOLEN\w.*|\bMANGANELL\w.*|(\bSPRAY.*\bPEPERONCINO\b))",
    r"((\bEUGENIA ROCCELLA\b|\bROCCELLA\b)|(\bNICOLA LAGIOIA\b|\bLAGIOIA\b)|(\bAUGUSTA MONTARULI\b|\bMONTARULI\b))",
    r"(\bATTACC\w.*\bGIORNALIST\w.*)|(\bGIORNALIST\w.*\bATTACCAT\w.*)",
    r"(\bG7\b)",
    r"(\bALBERTO ANGELA\b|\bDIVULGAZIONE SCIENTIFICA\b)",
    r"(\bCEMENTIFICA\w.*)|(\bOVIDIO MARRAS\b)",
    r"\bFAMIGLI\w.*?\bTRADIZIONAL\w.",
    r"(\bFIORAVANTI\b)|(\bVALERIO FIORAVANTI\b)|(\bSTRAGE DI BOLOGNA\b)",
    r"(\b2 GIUGNO\b)|(\bFESTA DELLA REPUBBLICA\b)",
    r"(\bMES\b)|(\bMECCANISMO EUROPEO DI STABILITà\b)",
    r"(\bSALUT\w.*?\bROMAN\w.*)|(\bDECIMA MAS\b)|(\bXMAS\b)|(\bX MAS\b)",
    r"(\bMASSIMO ZEN\b)|(\bLEGITTIMA DIFESA\b\bUCCI\w*)|(\bUCCI\w*\bLEGITTIMA DIFESA\b)",
    r"(\bABORT\w*)|(\bINTERR\w*\bGRAVIDANZ\w.*)"
]


VERSIONE COMPOUND

In [6]:
opinion_sentences = []
polarities = []
txt_col = df.iloc[:,38:].applymap(cleantxt) #colonne di testo
sent_df = deepcopy(txt_col)
for i in range(34): #per ogni colonna di testo
    entities_rgx = rgx[i] #definire pattern entità
    index = -1
    for t in txt_col.iloc[:,i]: #per ogni tweet della colonna di testo fissata (da vettorizzare)
        index += 1
        if pd.notna(t):
            opinion_sentences.clear() #svuota liste
            polarities.clear()
            text = t
            doc = nlp(text)
            for sentence in doc.sents:
                if re.search(entities_rgx.lower(), sentence.text.lower()):
                    opinion_sentences.append(sentence.text) #estrai frasi rispetto all'entità
                    
            for x in range(len(opinion_sentences)):
                opinion_sentence = opinion_sentences[x]
                polarities.append(list(analyzer.polarity_scores(opinion_sentence).values())[3])
            sent_df.iloc[index,i] = (sum(polarities)/len(opinion_sentences)) if opinion_sentences else 0  #calcola polarità totale tweet
        else:
            sent_df.iloc[index,i] = 0
            
    del entities_rgx
    del index

In [7]:
sent_df

,ANIMALI OR CUCCIOLI OR PICCOLI AND PRESI OR PORTATI object_all,GUERRA AND UCRAINA object_all,SOSTENIBILITà OR AUTO_ELETTRICHE OR AUTO_ELETTRICA object_all,VACCINI OR VAX OR NO_VAX OR SI_VAX object_all,FRATELLI AND ITALIA OR FDI object_all,FABIO_FAZIO OR FAZIO OR CHE_TEMPO_CHE_FA OR CHE_TEMPO_CHE_FU object_all,EUROVISION object_all,MISURE_ANTI_COVID OR MISURE_ANTICOVID OR GREENPASS OR GREEN_PASS OR TAMPONI object_all,LGBT OR LGBTQ OR LGBTQI OR LGBTQIA OR LGBTQIA+ OR LGBTQ+ OR LGBTQI+ object_all,GIORGIA_MELONI OR MELONI object_all,...,G7 object_all,ALBERTO_ANGELA OR DIVULGAZIONE_SCIENTIFICA object_all,CEMENTIFICAZIONE OR OVIDIO_MARRAS object_all,FAMIGLIA_TRADIZIONALE object_all,FIORAVANTI OR VALERIO_FIORAVANTI OR STRAGE_DI_BOLOGNA object_all,2_GIUGNO OR FESTA_DELLA_REPUBBLICA object_all,MES OR MECCANISMO_EUROPEO_DI_STABILITà object_all,SALUTO_ROMANO OR DECIMA_MAS OR XMAS OR X_MAS object_all,MASSIMO_ZEN OR LEGITTIMA_DIFESA AND UCCISO OR UCCIDERE object_all,ABORTO OR INTERRUZIONE_DI_GRAVIDANZA object_all
0,0,0,0.0,0,0,0.152,0,0.0,0.021,0.219,...,0,0,0,0.0,0,0.05,0,0.089667,0,0
1,0,0,0.0,0.0355,0,0.056333,0,0.07275,0,0.0915,...,0,0,0,0,0,0,0,0,0,0
2,0,0.0735,0,0,0,0.0,0.363,0.0,0.117,0.03825,...,0,0,0,0,0.0,0,0,0,0,0
3,0,0,0,0,0,0.0,0,0,0.0,0.191818,...,0,0,0,0,0.0,0,0,0,0,0
4,0,0,0.227,0,0.103,0,0.0,0,0.109,0.053444,...,0,0,0,0.0,0,0,0,0,0,0
5,0,0,0.0,0.054667,0,0.0,0,0.0,0.03675,0.0,...,0.0,0,0,0,0,0,0,0,0,0
6,0,0,0,0,0.126,0.0215,0.101,0,0.0,0.069,...,0,0,0,0,0,0,0,0,0,0
7,0,0,0,0,0.105,0.187,0,0,0,0.06575,...,0,0,0,0,0.266,0.171,0,0.0,0,0
8,0,0,0,0,0.077333,0.0485,0,0,0.054,0.043,...,0,0,0,0,0,0,0,0.0,0,0
9,0,0.094,0,0,0,0.0,0,0.079,0,0.0,...,0,0,0,0,0,0,0,0,0,0


In [9]:
index

NameError: name 'index' is not defined

In [10]:
df.iloc[:,38:] = sent_df

In [11]:
with pd.ExcelWriter("E:\Tesi Magistrale 2023\datasets\output\prima_sent\output_1_sent_v2.ods", engine='odf') as writer:
    df.to_excel(writer)  
